# Differentiation Products

### Setup

In [83]:
# Primitives
import numpy as np
import scipy as sp

def ownership(p2f):
    J = len(p2f)
    F = len(np.unique(p2f))
    Ω = np.zeros((J,J))
    for i in range(J):
        for j in range(J):
            if p2f[i] == p2f[j]:
                Ω[i,j] = 1
    return Ω  

def demand_derivatives(N, α, β):
    dqdp = β * np.ones((N,N))
    np.fill_diagonal(dqdp, α)
    return dqdp

def calibrateCost(p, q, dqdp, Ω):
    return p + np.dot(np.linalg.inv(Ω*dqdp.T),q)

def calibrateA(p, q, dqdp):
    return q - np.dot(dqdp,p)

def Q(a,p,dqdp):
    return a + np.dot(dqdp,p)

def P(a,q,dqdp):
    return np.dot(np.linalg.inv(dqdp).T,q - a)

def foc(p,c,α,β,Ω,a):
    dqdp = demand_derivatives(c.shape[0], α, β)
    q = Q(a,p,dqdp)
    FOC = -p+c-np.dot(np.linalg.inv(Ω*dqdp.T),q)
    return np.round(FOC,4)

def optimalP(c,α,β,Ω,a):
    p = sp.optimize.fsolve(foc, x0=np.zeros(c.shape[0]),args=(c,α,β,Ω,a))
    return p

def AvgP(p, q):
    return np.sum(p*q/np.sum(q))

def HHI(q, p2f):
    return np.sum(np.bincount(p2f, weights=q/np.sum(q))**2)

def UPP(p,q,c,α,β,Ω):
    dqdp = demand_derivatives(c.shape[0], α, β)
    UPP = c-np.dot(np.linalg.inv(Ω*dqdp.T),q)-p
    return np.round(UPP,4)

In [97]:
N = 4
α = -1.0
β = 0.1
p2f = np.arange(N)
Ω = ownership(p2f)
dqdp = demand_derivatives(N, α, β)
q = np.array([0.3,0.3,0.2,0.2])
p = np.array([1,1,1,1])
c = calibrateCost(p, q, dqdp, Ω)
a = calibrateA(p,q, dqdp) 
c, a

(array([0.7, 0.7, 0.8, 0.8]), array([1. , 1. , 0.9, 0.9]))

In [98]:
Q(a,p,dqdp)

array([0.3, 0.3, 0.2, 0.2])

In [99]:
P(a,q,dqdp)

array([1., 1., 1., 1.])

In [100]:
foc(p,c,α,β,Ω,a)

array([0., 0., 0., 0.])

In [101]:
optimalP(c,α,β,Ω,a)

array([1.00000155, 0.99998502, 1.00001068, 1.00001541])

In [102]:
AvgP(p, q)

1.0

In [103]:
HHI(q, p2f)

0.26

In [104]:
# cost increase
UPP(p,q,c+0.1,α,β,Ω)

array([0.1, 0.1, 0.1, 0.1])

In [105]:
# cross elasticity increase
UPP(p,q,c,α,β+0.3,Ω)

array([0., 0., 0., 0.])

In [106]:
# price sensitivity increase
UPP(p,q,c,α+0.1,β,Ω)

array([0.0333, 0.0333, 0.0222, 0.0222])

In [138]:
# Merger first 2
Ω_two = ownership(np.array([0,0,1,2]))
UPP(p,q,c,α,β,Ω_two)

array([0.0333, 0.0333, 0.    , 0.    ])

In [181]:
# Merger all four
Ω_all = ownership(np.array([0,0,0,0]))
UPP(p,q,c,α,β,Ω_all)

array([0.1026, 0.1026, 0.1117, 0.1117])

### Analysis

In [213]:
# Primitives
c = np.array([0.7, 0.7, 0.8, 0.8])
a = np.array([1. , 1. , 0.9, 0.9])
α = -1.0
β = 0.1
p2f = np.arange(N)

class industry:
    def __init__(self, c, α, β, p2f, a):
        self.c = c
        self.N = c.shape[0]
        self.α = α
        self.β = β
        self.a = a
        self.dqdp = demand_derivatives(c.shape[0], α, β)
        self.p2f = p2f
        self.Ω = ownership(p2f)
        self.p = optimalP(c,α,β,ownership(p2f),a)
        self.q = Q(self.a, self.p, self.dqdp)
        self.avgP = AvgP(self.p, self.q)
        self.hhi = HHI(self.q, self.p2f)
        self.ehhi = eHHI1(self.c, self.Ω,self.dqdp,self.q,self.α,self.β)

In [214]:
# Baseline
ind = industry(c, α, β, p2f, a)
ind.hhi, ind.avgP, ind.ehhi

(0.26000452477916414, 1.0000011882221, 8.008982925199948)

In [215]:
# Cost increase
ind = industry(c+0.1, α, β, p2f, a)
ind.hhi, ind.avgP, ind.ehhi

(0.26433102386525553, 1.058816926576504, 8.83502215262771)

In [216]:
# Own-Price sensitivity rises
ind = industry(c, α+0.1, β, p2f, a)
ind.hhi, ind.avgP, ind.ehhi

(0.25623142116102726, 1.083735415725671, 6.6564795600107445)

In [217]:
# Cross-Price sensitivity rises
ind = industry(c, α, β+0.1, p2f, a)
ind.hhi, ind.avgP, ind.ehhi

(0.2528993989051034, 1.2142790447989626, 4.8027685316491775)

In [218]:
# Baseline Demand rises
ind = industry(c, α, β, p2f, a+0.1)
ind.hhi, ind.avgP, ind.ehhi

(0.25655272323124684, 1.0588168299441134, 8.481938988097948)

In [219]:
# Full merger
ind = industry(c, α, β, np.array([0,0,0,0]), a)
ind.hhi, ind.avgP, ind.ehhi

(1.0, 1.053015211554366, 7.714723758829626)

In [220]:
# First two
ind = industry(c, α, β, np.array([0,0,1,2]), a)
ind.hhi, ind.avgP, ind.ehhi

(0.4288198350483916, 1.0106303597615456, 7.957256409621959)

### Comparing Cases

- (HHI, p, q) change due to change in primitives. 
- HHI can change due to c, alpha, beta, a, Omega. 
- We want EHHI to be a new metric based on HHI. "Effective HHI" 
- We want EHHI to be change ONLY when Omega changes. 
- We do not want EHHI to change when costs, alpha, beta, a change.
- We want to isolate the effect of Mergers only.